# 05 — Evaluation Metrics (Binary Classification)
**Goal:** Train a small classifier on synthetic data; compute Accuracy, Precision, Recall, F1; plot ROC curve.


In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
torch.manual_seed(0)

In [ ]:
# Data
X, y = make_classification(n_samples=2000, n_features=8, n_informative=4, n_redundant=0, random_state=0, weights=[0.6,0.4])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test_np = y_test  # keep numpy copy
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Model
class Clf(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(8,16), nn.ReLU(), nn.Linear(16,8), nn.ReLU(), nn.Linear(8,2))
    def forward(self,x): return self.net(x)

model = Clf()
opt = optim.Adam(model.parameters(), lr=0.01)
crit = nn.CrossEntropyLoss()

# Train
for ep in range(200):
    opt.zero_grad()
    out = model(X_train)
    loss = crit(out, y_train)
    loss.backward()
    opt.step()

# Predict
with torch.no_grad():
    logits = model(X_test)
    probs = torch.softmax(logits, dim=1)[:,1].numpy()
    preds = logits.argmax(1).numpy()

acc = accuracy_score(y_test_np, preds)
prec = precision_score(y_test_np, preds)
rec = recall_score(y_test_np, preds)
f1 = f1_score(y_test_np, preds)
fpr, tpr, _ = roc_curve(y_test_np, probs)
roc_auc = auc(fpr, tpr)

print(f'Accuracy={acc:.3f}, Precision={prec:.3f}, Recall={rec:.3f}, F1={f1:.3f}, AUC={roc_auc:.3f}')
plt.plot(fpr, tpr, label=f'ROC (AUC={roc_auc:.2f})'); plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.legend(); plt.title('ROC Curve'); plt.show()

## 🧩 Student Practice 1 — Confusion Matrix
- Compute and visualize the confusion matrix.

In [ ]:
# TODO: from sklearn.metrics import confusion_matrix; plot with matplotlib or seaborn


## ⚙️ Student Practice 2 — Class Imbalance
- Modify `weights` in `make_classification` (e.g., [0.9, 0.1]) and observe metric changes.

In [ ]:
# TODO: Recreate dataset with imbalance and compare accuracy vs F1
